#### Part 1: Data Preprocessing

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from pathlib import Path
import os

In [ ]:
# Define paths to your dataset
cwd = Path.cwd() / "dogs_cats"  # create your own path
train_path = os.path.join(cwd, 'training_set')  # path to your training dataset
test_path = os.path.join(cwd, 'test_set')  # path to your test dataset

In [ ]:
# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Create a single generator for both training and validation
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Use 'training' for the training set
)


Found 6404 images belonging to 1 classes.


In [ ]:
valid_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Use 'validation' for the validation set
)

Found 1601 images belonging to 1 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2023 images belonging to 1 classes.


#### Part 2 - making the CNN

In [ ]:
from tensorflow.keras.layers import Dropout

# Build the CNN model
model = Sequential()

model.add(Conv2D(4, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
# model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # 1 neuron for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=2,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size
)

Epoch 1/2
200/200 [==============================] - 46s 176ms/step - loss: 0.0157 - accuracy: 0.9920 - val_loss: 7.7958e-04 - val_accuracy: 1.0000
Epoch 2/2
200/200 [==============================] - 21s 105ms/step - loss: 1.0834e-04 - accuracy: 1.0000 - val_loss: 3.8115e-04 - val_accuracy: 1.0000


#### Part 3 - Making Predictions on new data

In [ ]:
def predict_new_image(img_path):  # Load an image for prediction
  img_path = os.path.join(cwd, img_path)  # Replace with the path to your image
  img = image.load_img(img_path, target_size=(64, 64))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0  # Rescale to match the training data

  # Make predictions
  predictions = model.predict(img_array)

  # Output the predictions
  if predictions[0][0] > 0.5:
      print("It's a dog")
  else:
      print("It's a cat")

In [ ]:
# put a testing image in your directory
# cat_image = "cat.jpg"
# predict_new_image(cat_image)

1/1 [==============================] - 0s 17ms/step
It's a cat
